In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
from scipy import interpolate
from scipy import spatial
import seaborn as sns

from utils import (config_rcparams,
                   set_axes_equal,
                   set_3d_params,
                   estimate_normals,
                   orient_normals)

In [ ]:
config_rcparams()

In [ ]:
%config InlineBackend.figure_format = 'retina'

# Toy example: Gaussian-pattern compliance-zone boundary

In [ ]:
def f(x, y, A=1, x0=0, y0=0, theta_x=1, theta_y=1):
    """2D Gaussian function.

    Parameters
    ----------
    x : float of numpy.ndarray
        Spatial coordinate(s), x-direction
    y : float of numpy.ndarray
        Spatial coordinate(s), y-direction
    A : float, optional
        Amplitude    
    x0 : float, optional
        Center of the blob, x-direction
    y0 : float, optional
        Center of the blob, y-direction
    theta_x : float, optional
        Spread of the blob, x-direction
    theta_y : float, optional
        Spread of the blob, y-direction

    Returns
    -------
    float or numpy.ndarray
        Value(s) of the Guassian function, z-direction
    """
    return A * np.exp(
        - (x - x0) ** 2 / (2 * theta_x ** 2)
        - (y - y0) ** 2 / (2 * theta_y ** 2))

In [ ]:
# generate surface points of the compliance-zone boundary
x = np.linspace(-1, 1, 51)
y = np.linspace(-1, 1, 51)
X, Y = np.meshgrid(x, y)
Z = f(X, Y, A=2, theta_x=0.3, theta_y=0.3)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax = set_3d_params(ax)
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)

## Normal estimation at each evaluation point

In [ ]:
# create the point cloud and generate a unit normal at each point
points = np.c_[X.ravel(), Y.ravel(), Z.ravel()]
normals = estimate_normals(points, k=20)
normals = orient_normals(points, normals, k=20)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax = set_3d_params(ax)
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)
q = ax.quiver(*points.T, *normals.T, color='k',
              lw=0.5, length=0.25, arrow_length_ratio=0.15)

## How to check whether a point is within the compliance-zone boundaries?

**Step 1** &ensp; Define the query point, $p$

**Step 2** &ensp; Find $k$ points on the compliance-zone boundary nearest to $p$ 

**Step 3** &ensp; Compute the scalar product between the relative position vector to $p$ from each of the $k$-nearest neighbors and the corresponding unit normal vector, $\mathbf{\hat{n}_i}$:

$$ \lvert \mathbf{p} - \mathbf{x_i} \rvert \cdot {\mathbf{\hat{n}_i}} $$

**Step 4** &ensp; Count the negative vs. positive values obtained in the previous step; if the ratio of the positive numbers is higher compared to the positive numbers, the point is located out of the compliance-zone boundaries

In [ ]:
# step 1
point_out = np.array([1, -1, 2])  # out of the compliance-zone boundaries

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
surf = ax.plot_surface(X, Y, Z, lw=0, cstride=1, rstride=1, antialiased=False)
ax.scatter(*points.T, fc='w', ec='k', s=5, lw=0.5)
ax.scatter(*point_out, fc='orange', ec='k', s=15, lw=0.5)
ax.text(*point_out + [0, 0, 0.2], f'{point_out}')
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# step 2
tree = spatial.KDTree(points)
dist, idx = tree.query(point_out, k=20)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax.scatter(*np.delete(points, idx, axis=0).T, fc='w', ec='k', s=5, lw=0.5)
ax.scatter(*points[idx, ...].T, fc='green', ec='k', s=15, lw=0.5)
ax.scatter(*point_out, fc='orange', ec='k', s=15, lw=0.5)
ax.text(*point_out + [0, 0, 0.2], f'{point_out}')
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax.scatter(*np.delete(points, idx, axis=0).T, fc='w', ec='k', s=5, lw=0.5)
ax.scatter(*points[idx, ...].T, fc='green', ec='k', s=15, lw=0.5)
ax.quiver(*points[idx, ...].T, *normals[idx, ...].T, color='k',
          lw=0.5, length=0.5, arrow_length_ratio=0.15)
ax.scatter(*point_out, fc='orange', ec='k', s=15, lw=0.5)
ax.scatter(0, 0, 0, fc='k', ec='k', s=15, lw=0.5)
ax.quiver(0, 0, 0, *point_out, color='k',
          lw=1, arrow_length_ratio=0.1)
ax.text(*point_out + [0, 0, 0.2], f'{point_out}')
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# step 3
prod = np.sum((point_out - points[idx]) * normals[idx], axis=1)

In [ ]:
# step 4
prob = np.sum(prod > 0) / prod.size
if prob > 0.5:
    print(f'The point is OUT of the compliance-zone boundaries ({prob:.2f})')
else:
    print(f'The point is WITHIN the compliance-zone boundaries ({1-prob:.2f})')

### A simple function to check whether a point is within the compliance-zone boundaries

The implementation is somewhat less robust as no SDF sampling is performed.
Instead, only the original points comprising the point cloud that represents the surface of the compliance-zone boundary are used.
This makes this method faster to execute and easier to implement, however there is no guarantee for convergence for complex boundary shapes as it is very sensitive on the number of examined closest points on the surface.

In [ ]:
def assess_compliance(query_points,
                      evaluation_points,
                      sample_count=5,
                      normals=None,
                      k=None,
                      **kwargs):
    """Return the indicator stating whether the query point is out of
    the compliance-zone boundary whose surface is represented as the
    unstructured point cloud.

    Parameters
    ----------
    query_points : numpy.ndarray
        Tested point(s) of shape (M, 3), M is the number of points
        being assessed for compliance
    evaluation_points : numpy.ndarray
        Point cloud of shape (N, 3), N is the number of points on the
        surface of the compliance-zone boundaryc
    sample_count : int, optional
        Number of closest points in the point cloud relative to each
        query point
    normals : numpy.ndarray, optional
        Normals of shape (N, 3), where N is the number of points in the
        point cloud. Normals should point out of the compliance zone
    k : float, optional
        Number of nearest neighbors for normal estimation
    **kwargs : dict, optional
        Additional keyword arguments for normal estimation if normals
        are not provided

    Returns
    -------
    numpy.ndarray 
        Of booleans indicating whether the query point is out of the 
    """    
    # handle points
    size = evaluation_points.shape[0]
    if size < 10:
        raise ValueError('Number of points must be > 10')  # for robustness

    # compute normals
    if normals is None:
        if not k:
            k = int(2 * np.log(size))
            if k < 5:
                k = 5
            elif k > 30:
                k = 30
        normals = estimate_normals(evaluation_points, k)
        normals = orient_normals(evaluation_points, normals, k)
    normals = normals / np.linalg.norm(normals, axis=1).reshape(-1, 1)

    # find points on the compliance-zone boundary closest to the query point
    tree = spatial.KDTree(evaluation_points)    
    dist, idx = tree.query(query_points, k=sample_count, workers=-1)
    closest_points = evaluation_points[idx]
    
    # compute the dot product between the relative position and normal vector
    pos_vec = np.atleast_2d(query_points)[:, np.newaxis, :] - closest_points

    # count the positive values indicating that the point is located outside
    out = np.einsum('ijk,ijk->ij', pos_vec, normals[idx]) > 0
    out = np.sum(out, axis=1) >= sample_count * 0.5
    return out

In [ ]:
# define query points to test an assessment function
query_points = np.c_[X.ravel(), Y.ravel(), np.ones_like(X).ravel()]

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax.contourf(X, Y, Z, zdir='y', offset=1, levels=1, colors='b')
ax.contourf(X, Y, Z, zdir='x', offset=-1, levels=1, colors='b')
ax.scatter(*query_points.T, fc='orange', ec='k', s=5, lw=0.5)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# check if the orange dots are inside or out of the compliance zone
out = assess_compliance(query_points, points,
                        sample_count=5,
                        normals=normals)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = plt.axes(projection='3d')
ax.contourf(X, Y, Z, zdir='y', offset=1, levels=1, colors='b')
ax.contourf(X, Y, Z, zdir='x', offset=-1, levels=1, colors='b')
ax.scatter(*query_points[out, ...].T, fc='w', ec='k', s=15, lw=0.5)
ax = set_3d_params(ax)
ax.view_init(25, -70);

In [ ]:
# find out the approximate radius where f(x, y) is ~1
idx = np.where(np.isclose(Z, 1, rtol=1e-2, atol=1e-2))
r = np.mean(np.sqrt(X[idx] ** 2 + Y[idx] ** 2))

In [ ]:
fig = plt.figure(figsize=(4, 4))
ax = plt.axes()
circle = patches.Circle((0, 0), r, fc='none', ec='k')
ax.add_patch(circle)
ax.scatter(*query_points[out, :2].T,
           fc='w', ec='k', s=7, lw=0.5)
ax.set(xlabel='x', ylabel='y');